# 기본 세팅

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# 필요한 라이브러리 임포트

import os
import torch
import random
import shutil
from google.colab import files
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate
from tqdm import tqdm

In [36]:
# 환경 세팅
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cpu


In [ ]:
# 모든 경로는 본인 환경에 맞춰 수정!

# 이미지 데이터셋 가져오기

## coco(content)
- 코랩 상에서 불러옴

In [4]:
# # 1. 다운로드 디렉토리 설정
# coco_drive_path = '/content/drive/MyDrive/Colab Notebooks/AIApp/images/coco'
# coco_local_path = '/content/test2017'
# os.makedirs(coco_drive_path, exist_ok=True)

# # 2. test2017.zip 다운로드
# !wget -O /content/test2017.zip http://images.cocodataset.org/zips/test2017.zip
# print("이미지 zip 다운로드 완료됨")

# # 3. 압축 해제 (unzip 사용)
# print("압축 해제 중...")
# !unzip -q /content/test2017.zip -d /content/
# print("압축 해제 완료")

# # 4. 이미지 파일 리스트 가져오기
# image_extensions = ('.jpg', '.jpeg', '.png')
# all_images = [f for f in os.listdir(coco_local_path) if f.lower().endswith(image_extensions)]
# print(f"전체 이미지 수: {len(all_images)}장")

# # 5. 랜덤하게 25,000장 추출
# random.seed(42)
# selected_images = random.sample(all_images, 25000)
# print("25,000장 랜덤 추출 완료")

# # 6. 선택한 이미지만 coco_drive_path에 복사
# for i, filename in enumerate(selected_images):
#     src_path = os.path.join(coco_local_path, filename)
#     dst_path = os.path.join(coco_drive_path, filename)
#     shutil.copy2(src_path, dst_path)
#     if (i+1) % 5000 == 0:
#         print(f"  ... {i+1}장 복사됨")
# print("이미지 복사 완료")

# # 7. 원본 압축파일과 전체 해제 폴더 제거
# os.remove('/content/test2017.zip')
# shutil.rmtree(coco_local_path)
# print("불필요한 파일 삭제 완료")

# # 8. 완료 메시지
# print(f"최종 저장 경로: {coco_drive_path}")
# final_images = [f for f in os.listdir(coco_drive_path) if f.lower().endswith(image_extensions)]
# print(f"최종 저장된 이미지 수: {len(final_images)}장")


--2025-05-20 20:35:29--  http://images.cocodataset.org/zips/test2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.132.57, 3.5.24.118, 52.216.221.241, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.132.57|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6646970404 (6.2G) [application/zip]
Saving to: ‘/content/test2017.zip’

/content/test2017.z 100%[===================>]   6.19G  56.4MB/s    in 1m 55s  

2025-05-20 20:37:24 (55.1 MB/s) - ‘/content/test2017.zip’ saved [6646970404/6646970404]

이미지 zip 다운로드 완료됨
압축 해제 중...
압축 해제 완료
전체 이미지 수: 40670장
25,000장 랜덤 추출 완료
  ... 5000장 복사됨
  ... 10000장 복사됨
  ... 15000장 복사됨
  ... 20000장 복사됨
  ... 25000장 복사됨
이미지 복사 완료
불필요한 파일 삭제 완료
최종 저장 경로: /content/drive/MyDrive/Colab Notebooks/AIApp/images/coco
최종 저장된 이미지 수: 25000장


## wikiart(style)
- https://www.kaggle.com/datasets/trungit/wikiart25k 에서 데이터셋 다운 받아 txt 파일 제거 후 압축본 구글 드라이브에 업로드 한 뒤 실행

In [12]:
# # zip 파일 압축 해제
# %cd /content/drive/MyDrive/Colab Notebooks/AIApp/images
# !unzip -qq "/content/drive/MyDrive/Colab Notebooks/AIApp/wikiart.zip"

/content/drive/MyDrive/Colab Notebooks/AIApp/images


In [15]:
# # 압축이 풀린 최상위 폴더 경로
# root_path = "/content/drive/MyDrive/Colab Notebooks/AIApp/images/wikiart"

# # 이미지 확장자 목록
# image_extensions = ('.jpg', '.jpeg', '.png')

# # 이미지 수 카운트
# image_count = 0
# for root, dirs, files in os.walk(root_path):
#     for file in files:
#         if file.lower().endswith(image_extensions):
#             image_count += 1

# print(f"총 이미지 수: {image_count}장")


🖼️ 총 이미지 수: 25000장


# VGG 인코더

In [22]:
# VGG-19 pretrained 모델에서 features만 가져오기
vgg = models.vgg19(pretrained=True).features

# relu1_1 ~ relu4_1까지 레이어 분할
class Net(nn.Module):
    def __init__(self, encoder):
        super(Net, self).__init__()
        enc_layers = list(encoder.children())

        # relu1_1, relu2_1, relu3_1, relu4_1까지만 사용
        self.enc_1 = nn.Sequential(*enc_layers[:4])    # input -> relu1_1
        self.enc_2 = nn.Sequential(*enc_layers[4:11])  # relu1_1 -> relu2_1
        self.enc_3 = nn.Sequential(*enc_layers[11:18]) # relu2_1 -> relu3_1
        self.enc_4 = nn.Sequential(*enc_layers[18:31]) # relu3_1 -> relu4_1

        # encoder 파라미터 고정
        for name in ['enc_1', 'enc_2', 'enc_3', 'enc_4']:
            for param in getattr(self, name).parameters():
                param.requires_grad = False

    # 중간 결과들 (relu1_1 ~ relu4_1) 반환
    def encode_with_intermediate(self, input):
        results = [input]
        for i in range(4):
            func = getattr(self, f'enc_{i + 1}')
            results.append(func(results[-1]))
        return results[1:]  # input 제외한 결과만 반환

    # relu4_1 결과만 반환
    def encode(self, input):
        for i in range(4):
            input = getattr(self, f'enc_{i + 1}')(input)
        return input

# 네트워크 객체 생성
net = Net(vgg).to('cuda' if torch.cuda.is_available() else 'cpu')


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:08<00:00, 69.2MB/s]


In [23]:
# 이미지 전처리

# 짧은 변 길이 변경
class ResizeShortSide:
    def __init__(self, size):
        self.size = size

    def __call__(self, img):
        w, h = img.size
        if w < h:
            new_w = self.size
            new_h = int(h * self.size / w)
        else:
            new_h = self.size
            new_w = int(w * self.size / h)
        return img.resize((new_w, new_h), Image.BICUBIC)

# 전체 전처리
vgg_transform = transforms.Compose([
    ResizeShortSide(512),
    transforms.RandomCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])


In [24]:
# 특징 추출

# 이미지 파일 확인
def is_image_file(filename):
    return filename.lower().endswith(('jpg', 'jpeg', 'png'))

# 하위 폴더 확인
class ImageFolderDataset(Dataset):
    def __init__(self, folder_path, transform, has_subfolders=True):
        self.transform = transform
        self.image_paths = []

        if has_subfolders:
            subfolders = [os.path.join(folder_path, d)
                          for d in os.listdir(folder_path)
                          if os.path.isdir(os.path.join(folder_path, d))]
        else:
            subfolders = [folder_path]

        for subfolder in subfolders:
            for fname in os.listdir(subfolder):
                if is_image_file(fname):
                    self.image_paths.append(os.path.join(subfolder, fname))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        try:
            image = Image.open(path).convert('RGB')
            tensor = self.transform(image)
            return tensor, path
        except Exception as e:
            print(f"[오류] 이미지 로딩 실패: {path}, 오류: {e}")
            return None  # collate_fn에서 필터링됨

# collate_fn 정의 (예외 이미지 무시)
def safe_collate(batch):
    batch = [b for b in batch if b is not None]
    if not batch:
        return None
    return default_collate(batch)

# 특징 추출 함수 (배치 기반)
def extract_features_batch(folder_path, transform, has_subfolders=True, batch_size=32, num_workers=2):
    dataset = ImageFolderDataset(folder_path, transform, has_subfolders)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False,
                            num_workers=num_workers, collate_fn=safe_collate)

    features = []
    paths = []

    for batch in tqdm(dataloader, desc=f"Extracting features from {os.path.basename(folder_path)}"):
        if batch is None:
            continue
        inputs, img_paths = batch
        inputs = inputs.to(device)

        with torch.no_grad():
            batch_features = net.encode(inputs)

        features.extend(batch_features.cpu())
        paths.extend(img_paths)

    return features, paths

In [26]:
# 테스트용 coco_test 만들기
coco_dir = '/content/drive/MyDrive/Colab Notebooks/AIApp/images/coco'
coco_test_dir = '/content/drive/MyDrive/Colab Notebooks/AIApp/images/coco_test'

# os.makedirs(coco_test_dir, exist_ok=True)

# image_files = sorted([
#     f for f in os.listdir(coco_dir)
#     if f.lower().endswith(('.jpg', '.jpeg', '.png'))
# ])[:32]

# # 복사 실행
# for fname in image_files:
#     src_path = os.path.join(coco_dir, fname)
#     dst_path = os.path.join(coco_test_dir, fname)
#     shutil.copy2(src_path, dst_path)

# print(f"{len(image_files)}개 이미지가 {coco_test_dir}로 복사되었습니다.")

32개 이미지가 /content/drive/MyDrive/Colab Notebooks/AIApp/images/coco_test로 복사되었습니다.


In [27]:
# 인코딩(테스트용)
coco_test_features, coco_test_paths = extract_features_batch(coco_test_dir, vgg_transform, has_subfolders=False)
coco_test_features[0].shape

Extracting features from coco_test: 100%|██████████| 1/1 [00:36<00:00, 36.55s/it]


torch.Size([512, 16, 16])

In [31]:
# # 인코딩(실제 사용) - 엄청 오래 걸림
# coco_dir = '/content/drive/MyDrive/Colab Notebooks/AIApp/images/coco'
# wikiart_dir = '/content/drive/MyDrive/Colab Notebooks/AIApp/images/wikiart'

# coco_features, coco_paths = extract_features_batch(coco_dir, vgg_transform, has_subfolders=False)
# print("coco 인코딩 끝")
# wikiart_features, wikiart_paths = extract_features_batch(wikiart_dir, vgg_transform, has_subfolders=True)
# print("wikiart 인코딩 끝")


# 텍스트 피쳐 변환

In [32]:
# 모델 정의 및 가중치 불러오기
num_classes = 4  # 감정 클래스 수
# image_dir = '/content/drive/MyDrive/Colab Notebooks/AIApp/images/coco_test' # 테스트용
image_dir = '/content/drive/MyDrive/Colab Notebooks/AIApp/images/coco' # 실제 사용

# 모델 정의 (학습 당시와 동일하게)
model = models.resnet18(pretrained=False)
in_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(in_features, num_classes)
)
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/AIApp/best_emotion_model_0520.pth',
                                 map_location=torch.device('cpu'))) # 원래 gpu로 불러와야되는데 지금 gpu 못 써서 임시로 cpu
model = model.to(device)
model.eval()

# 전처리 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [33]:
# 이미지 예측하여 label_idx 추출
predicted_indices = []

image_paths = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir)
               if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]

for path in tqdm(image_paths):
    image = Image.open(path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        _, pred = torch.max(output, 1)
        predicted_indices.append(pred.item())

preds_tensor = torch.tensor(predicted_indices)
preds_tensor

100%|██████████| 25000/25000 [1:03:21<00:00,  6.58it/s]


tensor([3, 2, 2,  ..., 2, 1, 2])

In [10]:
# 임베딩, 선형 변환
embedding_dim = 512

# 임베딩 & 선형변환 정의
class_embedding = nn.Embedding(num_classes, embedding_dim)
linear = nn.Linear(embedding_dim, 512)

# GPU에 올리기 (선택사항)
class_embedding = class_embedding.to(device)
linear = linear.to(device)

# 변환 수행
embedded = class_embedding(preds_tensor.to(device))
final_output = linear(embedded)
